<a href="https://colab.research.google.com/github/juanfrans-courses/threads-storytelling-with-maps-and-data/blob/main/04_Public_Data/Data_Part_2_Advanced_Census_Querying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Part 2 - Advanced Census Querying

**Threads - Storytelling with Maps and Data (Spring 2025)**

## Setting Up

Before we install our packages and start to work with census data, we need to register for a Census API Key so that we can remove the majority of restrictions when it comes to accessing and downloading Census Data.

To do so, go to [api.census.gov/data/key_signup.html](https://api.census.gov/data/key_signup.html) and provide your basic access information (name and organization).

Check your email and **make sure you click on the link** that says "click here to activate your key".

Store your API key in a place familiar to you, like a password manager.

Now let's import the normal libraries plus [census](https://pypi.org/project/census/) and [jenkspy](https://pypi.org/project/jenkspy/).

In [ ]:
!pip install census
!pip install jenkspy
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
from census import Census
import jenkspy

It is always good practice to save your keys and secrets in your local environment. That way when you push code to a public repository those secrets or keys don't become public. In Colab you can replicate this by adding them to your secrets in the left hand panel and then loading them like this:

In [ ]:
from google.colab import userdata
CENSUS_API_KEY = userdata.get('CENSUS_API_KEY')

## Querying Census Data With Census API

Now let's use the census API library to get data from the census. We will use this data to create three maps, all for New York City census tracts:

* Median household income
* Gini coefficient
* Rent as percent of income

All three maps should include the census tract's total population in the tooltip.

The corresponding ACS 2020 variables are:

* `B01003_001E`: total population
* `B19013_001E`: median household income
* `B19083_001E`: gini coefficient
* `B25071_001E`: rent as percent of income

First, let's create a variable that holds our "Census" object with our API key.

In [ ]:
c = Census(CENSUS_API_KEY)

Next, let's get the data for the first map: total population and median household income. The syntax for the query is actually pretty simple: here we are getting three variables from the 2020 ACS 5-year survey at the census tract level for the **state** of New York. Further down we will filter this to get just the tracts for New York City. Here is the list of corresponding [FIPS codes for US states](https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html).

In [ ]:
data = c.acs5.get(
    ('NAME', 'B01003_001E', 'B19013_001E'),  # Variables: census tract name, total population, median household income
    {'for': 'tract:*', 'in': 'state:36'},  # Geographic filters: All census tracts in New York State (code 36)
    year=2020,
)

The result is a list with all the data. Let's print the first five items in the list.

In [ ]:
data[:5]

And now let's convert that into a Pandas DataFrame. This will make it easier to clean it and to join it with a GeoDataFrame of the census tract geometries.

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

Now we need to get the tracts just for New York City. We do this by selecting only the ones for New York City counties. Again, we use the [corresponding FIPS codes](https://www.census.gov/library/reference/code-lists/ansi.html#cou). Remember, New York City counties are Bronx, Kings, New York, Queens and Richmond.

In [ ]:
new_york_counties = ['005', '047', '061', '081', '085']

In [ ]:
nyc_df = df[df['county'].isin(new_york_counties)].copy(deep=True)

In [ ]:
nyc_df.shape

Now that we have the data just for New York City's census tracts, let's bring in a shapefile with the corresponding census tract geographies as a GeoDataFrame. We will then join the census data we donwloaded to this GeoDataFrame.

There are multiple sites where you can find geographic files, even within the census page:

* You can find all the census geographies in the [Census TIGER page](https://www.census.gov/cgi-bin/geo/shapefiles/index.php).
* You can also find simplified versions of most of these files in the [Cartographic Boundaries](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.2020.html#list-tab-1883739534) section. When possible, I recommend using this over the main TIGER page. These files are lighter and more suitable to mapping.

But, for our purposes, we'll actually use the [Bytes of the Big Apple](https://www.nyc.gov/site/planning/data-maps/open-data/census-download-metadata.page) page from NYC's Department of City Planning because they provide census tracts clipped to the shoreline.

Always make sure you donwload the geography for the year that corresponds to the data you downloaded.

In [ ]:
# gdf = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_36_tract.zip') # Main TIGER page
# gdf = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_36_tract_500k.zip') # Cartographic Boundaries
gdf = gpd.read_file('https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Census_Tracts_for_2020_US_Census/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson') # Bytes of the Big Apple NYC Department of City Planning

In [ ]:
gdf.head()

Now we need to join the two datasets using a common column. Notice that the geography one has a column called `GEOID`. This column is a code made out of the state code, the county code and the census tract code. Let's then build that same code for the data we downloaded so we can use it to join the two datasets.

In [ ]:
nyc_df['GEOID'] = nyc_df['state'] + nyc_df['county'] + nyc_df['tract']

Now let's merge them using the `GEOID` field.

In [ ]:
nyc_gdf = gdf.merge(nyc_df, on='GEOID')

In [ ]:
nyc_gdf.head()

In [ ]:
len(nyc_gdf)

Finally, let's filter out the census tracts where there is no income.

In [ ]:
nyc_gdf = nyc_gdf[nyc_gdf['B19013_001E'] > 0]

And let's create the map.

In [ ]:
mnIncomeMap = alt.Chart(nyc_gdf).mark_geoshape().encode(
    color=alt.Color('B19013_001E:Q', legend=alt.Legend(title='MHHI per Census Tract (Dollars)', format='$,.2r'), scale=alt.Scale(scheme='purples', domain=[0, 250000])),
    tooltip=[alt.Tooltip('NAME',title='County'),
      alt.Tooltip('B01003_001E',title='Population', format=',.2r'),
      alt.Tooltip('B19013_001E',title='Median Household Income', format='$,.2r')]
).properties(
    width=700,
    height=750,
    title='Median Household Income (New York City)'
).project(
    type='mercator'
)

mnIncomeMap

## In Class Excercise

Go ahead and build the next two maps.